In [ ]:

import numpy as np
#import matplotlib
import matplotlib.pyplot as plt
#import argparse
import cv2
import time
import pixellib
from pixellib.instance import instance_segmentation

In [ ]:


#  video to be processed
cap = cv2.VideoCapture('Ltest3.mp4')
while(cap.isOpened()):

    # read video frame & show on screen
    ret, frame = cap.read()
    # cv2.imshow("Original Scene", frame)

    # snip section of video frame of interest & show on screen
    snip = frame[0:1080,0:1920]
    #cv2.imshow("Snip",snip)
    #instance segmentation
    segmentation_model = instance_segmentation()
    segmentation_model.load_model('mask_rcnn_coco.h5')
    res= segmentation_model.segmentFrame(frame,show_bboxes=True)
    display = res[1]
    cv2.imshow("Black/White", display)
    # create polygon (trapezoid) mask to select region of interest
    mask = np.zeros((snip.shape[0], snip.shape[1]), dtype="uint8")
    pts = np.array([[0, 797], [680, 370],[1920, 1073], [1400, 620]], dtype=np.int32)
    cv2.fillConvexPoly(mask, pts, 255)
   
    cv2.imshow("Mask", mask)
    print(snip.shape)
    # apply mask and show masked image on screen
    masked = cv2.bitwise_and(snip, snip, mask=mask)
    #plt.imshow(masked)
    #plt.show()

   
    frame = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)
    thresh =195
    frame = cv2.threshold(frame, thresh, 255, cv2.THRESH_BINARY)[1]
    cv2.imshow("Black/White", frame)

    
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    # cv2.imshow("Blurred", blurred)

   
    edged = cv2.Canny(blurred, 170, 220)
    cv2.imshow("Edged", edged)


    lines = cv2.HoughLines(edged, 1, np.pi / 180, 60, lines=np.array([]))

    # defining arrays for left and right lanes
    rho_left = []
    theta_left = []
    rho_right = []
    theta_right = []

    # ensure cv2.HoughLines found at least one line
    if lines is not None:

        # loop through all of the lines found by cv2.HoughLines
        for i in range(0, len(lines)):

            # evaluate each row of cv2.HoughLines output 'lines'
            for rho, theta in lines[i]:

                # collect left lanes
                if theta < np.pi/2 and theta > np.pi/4:
                    rho_left.append(rho)
                    theta_left.append(theta)


                # collect right lanes
                if theta > np.pi/2 and theta < 3*np.pi/4:
                    rho_right.append(rho)
                    theta_right.append(theta)

    # statistics to identify median lane dimensions
    left_rho = np.average(rho_left)
    left_theta = np.average(theta_left)
    right_rho = np.average(rho_right)
    right_theta = np.average(theta_right)

    # plot median lane on top of scene snip
    if left_theta > np.pi/4:
        a = np.cos(left_theta); b = np.sin(left_theta)
        x0 = a * left_rho; y0 = b * left_rho
        offset1 = 200; offset2 = 300
        x1 = int(x0 - offset1 * (-b)); y1 = int(y0 - offset1 * (a))
        x2 = int(x0 + offset2 * (-b)); y2 = int(y0 + offset2 * (a))

        cv2.line(snip, (x1, y1), (x2, y2), (0, 255, 0), 6)

    if right_theta > np.pi/4:
        a = np.cos(right_theta); b = np.sin(right_theta)
        x0 = a * right_rho; y0 = b * right_rho
        offset1 = 2050; offset2 = 830
        x3 = int(x0 - offset1 * (-b)); y3 = int(y0 - offset1 * (a))
        x4 = int(x0 - offset2 * (-b)); y4 = int(y0 - offset2 * (a))

        cv2.line(snip, (x3, y3), (x4, y4), (0, 255, 0), 6)




    if left_theta > np.pi/4 and right_theta > np.pi/4:
        pts = np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], dtype=np.int32)


        overlay = snip.copy()

        cv2.fillConvexPoly(overlay, pts, (255, 100, 0))

        opacity = 0.2
        cv2.addWeighted(overlay, opacity, snip, 1 - opacity, 0, snip)
    
    
    cv2.imshow("res",snip)
    cv2.waitKey(1)



    if cv2.waitKey(10) & 0xFF == ord('q'):
        break



In [ ]:
cap.release()
cv2.destroyAllWindows()